<a href="https://colab.research.google.com/github/Taruna-07/Project_Phase/blob/main/Copy_of_Design_and_analysis_of_ML_model_for_infection_detection_in_lung_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Design ML model to detect infection in lung***

1.Importing Tensorflow version and if the version in less than 2.0.0 then install the 2.0.0 version using pip install

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
#!pip install tensorflow-gpu==2.0.0

2.Importing Tensorflow libraries

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt

3. Syntax for creating white background image

In [ ]:
# Create images with white backgrounds
import plotly.io as pio
pio.templates.default = 'plotly_white'

4. Connecting Google drive with google colab to extract dataset

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
fid = drive.ListFile({'q':"title='PneumoniaCXR.zip'"}).GetList()[0]['id']
f = drive.CreateFile({'id': fid})
f.GetContentFile('PneumoniaCXR.zip')

In [ ]:
f.keys()

5.Extracting dataset from zip file

In [ ]:
!unzip PneumoniaCXR.zip

The file is archived as a ZIP file in Google Drive.

Note the original folder structure

Train

*   Viral

*   Bacterial

*   Normal

Test

*   Viral

*   Bacterial

*   Normal



Make sure that the sub-folders have exactly the same names (and cases, i.e. each sub-folder name starts with an uppercase letter).

The images were originally stored in the structure above inside a folder named PneumoniaCXR. This folder was compressed as a ZIP file and uploaded to Google Drive.

In [ ]:
PATH = '/content/PneumoniaCXR'

In [ ]:
os.listdir(PATH)

6.Inspecting path and dimension of data

In [ ]:
train_dir = os.path.join(PATH,
                         'Train')
test_dir = os.path.join(PATH,
                        'Test')

In [ ]:
train_normal_dir = os.path.join(train_dir,
                                'Normal')
train_bacterial_dir = os.path.join(train_dir,
                                   'Bacterial')
train_viral_dir = os.path.join(train_dir,
                               'Viral')

test_normal_dir = os.path.join(test_dir,
                               'Normal')
test_bacterial_dir = os.path.join(test_dir,
                                  'Bacterial')
test_viral_dir = os.path.join(test_dir,
                              'Viral')

In [ ]:
len(os.listdir(train_normal_dir))

In [ ]:
len(os.listdir(test_viral_dir))

In [ ]:
len(os.listdir(test_bacterial_dir))

7.Preparing the data

In [ ]:
IMG_HEIGHT = 212
IMG_WIDTH = 212
batch_size = 8

In [ ]:
image_gen = ImageDataGenerator(
    rescale=1./255
    )

In [ ]:
train_data_gen = image_gen.flow_from_directory(
    batch_size=batch_size,
    directory=train_dir,
    shuffle=True,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical')

In [ ]:
test_data_gen = image_gen.flow_from_directory(
    batch_size=batch_size,
    directory=test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    class_mode='categorical')

In [ ]:
sample_training_images, _ = next(train_data_gen)

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 3, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        ax.imshow(img)
        ax.axis('off')
    plt.tight_layout()
    plt.show()

In [ ]:
plotImages(sample_training_images[:3])

8.Creating Neural network

In [ ]:
model = Sequential([
    Conv2D(16, 3, padding='same', activation='relu', input_shape=(IMG_HEIGHT, IMG_WIDTH, 3)),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(32, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Conv2D(64, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Conv2D(128, 3, padding='same', activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(3, activation='softmax')
])

In [ ]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

9.Training the model

In [ ]:
batch_size = 8
epochs = 10

In [ ]:
num_normal_train = len(os.listdir(train_normal_dir))
num_bacterial_train = len(os.listdir(train_bacterial_dir))
num_viral_train = len(os.listdir(train_viral_dir))

num_normal_test = len(os.listdir(test_normal_dir))
num_bacterial_test = len(os.listdir(test_bacterial_dir))
num_viral_test = len(os.listdir(test_viral_dir))

total_train = num_normal_train + num_bacterial_train + num_viral_train
total_test = num_normal_test + num_bacterial_test + num_viral_test

In [ ]:
history = model.fit_generator(
    train_data_gen,
    steps_per_epoch=total_train // batch_size,
    epochs=epochs,
    validation_data=test_data_gen,
    validation_steps=total_test // batch_size,
    callbacks = [tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        min_delta=0.01,
        patience=3
    )]
)

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(x=history.epoch,
                         y=history.history['accuracy'],
                         mode='lines+markers',
                         name='Training accuracy'))
fig.add_trace(go.Scatter(x=history.epoch,
                         y=history.history['val_accuracy'],
                         mode='lines+markers',
                         name='Validation accuracy'))
fig.update_layout(title='Accuracy',
                  xaxis=dict(title='Epoch'),
                  yaxis=dict(title='Percentage'))
fig.show()

10.Saving model

In [ ]:
save_path = F'/content/gdrive/My Drive/Design and analysis of ML model for infection detection in lung/'

In [ ]:
model.save_weights(os.path.join(PATH, 'CXR.h5'))

In [ ]:
model_json = model.to_json()

In [ ]:
with open(os.path.join(PATH, 'CXR.json'), 'w') as json_file:
  json_file.write(model_json)

In [ ]:
json_file.close()

11.Transfer Learning

In [ ]:
IMG_HEIGHT = 128
IMG_WIDTH = 128

In [ ]:
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, 3)

# Create the base model from the pre-trained model MobileNet V2
base_model = tf.keras.applications.MobileNetV2(input_shape=IMG_SHAPE,
                                               include_top=False,
                                               weights='imagenet')

In [ ]:
base_model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False